###To run it directly on huggingface interface
from llama import BasicModelRunner

model = BasicModelRunner("EleutherAI/pythia-410m")

model.load_data_from_jsonlines("lamini_docs.jsonl", input_key="question", output_key="answer")

model.train(is_public=True)

In [2]:
!pip install lamini

In [40]:
import os
import lamini

lamini.api_url = os.getenv("POWERML__PRODUCTION__URL")
lamini.api_key = "Get api Key

In [4]:
import datasets
import tempfile
import logging
import random
import config
import os
import yaml
import time
import torch
import transformers
import pandas as pd
import jsonlines

from utilities import *
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import TrainingArguments
from transformers import AutoModelForCausalLM
from llama import BasicModelRunner


logger = logging.getLogger(__name__)
global_config = None

In [5]:
dataset_name = "lamini_docs.jsonl"
dataset_path = f"/content/{dataset_name}"
use_hf = False

In [6]:
dataset_path = "lamini/lamini_docs"
use_hf = True

In [7]:
model_name = "EleutherAI/pythia-70m"

In [8]:
training_config = {
    "model": {
        "pretrained_name": model_name,
        "max_length" : 2048
    },
    "datasets": {
        "use_hf": use_hf,
        "path": dataset_path
    },
    "verbose": True
}

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
train_dataset, test_dataset = tokenize_and_split_data(training_config, tokenizer)

print(train_dataset)
print(test_dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
DEBUG:utilities:Config: datasets.path: lamini/lamini_docs
datasets.use_hf: true
model.max_length: 2048
model.pretrained_name: EleutherAI/pythia-70m
verbose: true



tokenize True lamini/lamini_docs


DEBUG:fsspec.local:open file: /root/.cache/huggingface/datasets/lamini___lamini_docs/default/0.0.0/05bd680b81d69a7a1d38193873f1487d73e535bf/dataset_info.json
DEBUG:fsspec.local:open file: /root/.cache/huggingface/datasets/lamini___lamini_docs/default/0.0.0/05bd680b81d69a7a1d38193873f1487d73e535bf/dataset_info.json


Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1260
})
Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 140
})


In [10]:
base_model = AutoModelForCausalLM.from_pretrained(model_name)

In [11]:
device_count = torch.cuda.device_count()
if device_count > 0:
    logger.debug("Select GPU device")
    device = torch.device("cuda")
else:
    logger.debug("Select CPU device")
    device = torch.device("cpu")

DEBUG:__main__:Select CPU device


In [12]:
base_model.to(device)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (a

In [13]:
def inference(text, model, tokenizer, max_input_tokens=1000, max_output_tokens=100):
  # Tokenize
  input_ids = tokenizer.encode(
          text,
          return_tensors="pt",
          truncation=True,
          max_length=max_input_tokens
  )

  # Generate
  device = model.device
  generated_tokens_with_prompt = model.generate(
    input_ids=input_ids.to(device),
    max_length=max_output_tokens
  )

  # Decode
  generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)

  # Strip the prompt
  generated_text_answer = generated_text_with_prompt[0][len(text):]

  return generated_text_answer

In [14]:
test_text = test_dataset[0]['question']
print("Question input (test):", test_text)
print(f"Correct answer from Lamini docs: {test_dataset[0]['answer']}")
print("Model's answer: ")
print(inference(test_text, base_model, tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question input (test): Can Lamini generate technical documentation or user manuals for software projects?
Correct answer from Lamini docs: Yes, Lamini can generate technical documentation and user manuals for software projects. It uses natural language generation techniques to create clear and concise documentation that is easy to understand for both technical and non-technical users. This can save developers a significant amount of time and effort in creating documentation, allowing them to focus on other aspects of their projects.
Model's answer: 


I have a question about the following:

How do I get the correct documentation to work?

A:

I think you need to use the following code:

A:

You can use the following code to get the correct documentation.

A:

You can use the following code to get the correct documentation.

A:

You can use the following


###Actual Training


In [27]:
max_steps = 50

In [28]:
trained_model_name = f"lamini_docs_{max_steps}_steps"
output_dir = trained_model_name

In [29]:
training_args = TrainingArguments(

  # Learning rate
  learning_rate=1.0e-5,

  # Number of training epochs
  num_train_epochs=3,

  # Max steps to train for (each step is a batch of data)
  # Overrides num_train_epochs, if not -1
  max_steps=max_steps,

  # Batch size for training
  per_device_train_batch_size=1,

  # Directory to save model checkpoints
  output_dir=output_dir,

  # Other arguments
  overwrite_output_dir=False, # Overwrite the content of the output directory
  disable_tqdm=False, # Disable progress bars
  eval_steps=120, # Number of update steps between two evaluations
  save_steps=120, # After # steps model is saved
  warmup_steps=1, # Number of warmup steps for learning rate scheduler
  per_device_eval_batch_size=1, # Batch size for evaluation
  evaluation_strategy="steps",
  logging_strategy="steps",
  logging_steps=1,
  optim="adafactor",
  gradient_accumulation_steps = 4,
  gradient_checkpointing=False,

  # Parameters for early stopping
  load_best_model_at_end=True,
  save_total_limit=1,
  metric_for_best_model="eval_loss",
  greater_is_better=False
)

In [30]:
#understanding memory footprint of the model
model_flops = (
  base_model.floating_point_ops(
    {
       "input_ids": torch.zeros(
           (1, training_config["model"]["max_length"])
      )
    }
  )
  * training_args.gradient_accumulation_steps
)

print(base_model)
print("Memory footprint", base_model.get_memory_footprint() / 1e9, "GB")
print("Flops", model_flops / 1e9, "GFLOPs")

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (a

In [31]:
trainer = Trainer(
    model=base_model,
    model_flops=model_flops,
    total_steps=max_steps,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(


In [20]:
#!pip install transformers==4.32.1

In [32]:
training_output = trainer.train()

Step,Training Loss,Validation Loss


DEBUG:utilities:Step (1) Logs: {'loss': 3.1545, 'learning_rate': 1e-05, 'epoch': 0.0, 'iter_time': 0.0, 'flops': 0.0, 'remaining_time': 0.0}
DEBUG:utilities:Step (2) Logs: {'loss': 2.7317, 'learning_rate': 9.795918367346939e-06, 'epoch': 0.01, 'iter_time': 11.357940673828125, 'flops': 193315661298.6572, 'remaining_time': 545.18115234375}
DEBUG:utilities:Step (3) Logs: {'loss': 3.0012, 'learning_rate': 9.591836734693878e-06, 'epoch': 0.01, 'iter_time': 8.397529482841492, 'flops': 261465924810.19153, 'remaining_time': 394.6838856935501}
DEBUG:utilities:Step (4) Logs: {'loss': 3.0613, 'learning_rate': 9.387755102040818e-06, 'epoch': 0.01, 'iter_time': 7.154932578404744, 'flops': 306874703331.103, 'remaining_time': 329.1268986066182}
DEBUG:utilities:Step (5) Logs: {'loss': 3.2148, 'learning_rate': 9.183673469387756e-06, 'epoch': 0.02, 'iter_time': 6.377653062343597, 'flops': 344275204513.1093, 'remaining_time': 286.9943878054619}
DEBUG:utilities:Step (6) Logs: {'loss': 2.7848, 'learning_ra

In [33]:
save_dir = f'{output_dir}/final'

trainer.save_model(save_dir)
print("Saved model to:", save_dir)

Saved model to: lamini_docs_50_steps/final


In [34]:
finetuned_slightly_model = AutoModelForCausalLM.from_pretrained(save_dir, local_files_only=True)


In [35]:
finetuned_slightly_model.to(device)


GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (a

In [36]:
test_question = test_dataset[0]['question']
print("Question input (test):", test_question)

print("Finetuned slightly model's answer: ")
print(inference(test_question, finetuned_slightly_model, tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question input (test): Can Lamini generate technical documentation or user manuals for software projects?
Finetuned slightly model's answer: 
Yes, Lamini can generate technical documentation or user manuals for software projects. Lamini can generate technical documentation or user manuals for software projects. Lamini can generate technical documentation or user manuals for software projects. Lamini can generate technical documentation or user manuals for software projects. Lamini can generate technical documentation or user manuals for software projects. Lamini can generate technical documentation or


In [26]:
test_answer = test_dataset[0]['answer']
print("Target answer output (test):", test_answer)

Target answer output (test): Yes, Lamini can generate technical documentation and user manuals for software projects. It uses natural language generation techniques to create clear and concise documentation that is easy to understand for both technical and non-technical users. This can save developers a significant amount of time and effort in creating documentation, allowing them to focus on other aspects of their projects.


In [41]:
#larger model
bigger_finetuned_model = BasicModelRunner(model_name_to_id["bigger_model_name"])
bigger_finetuned_output = bigger_finetuned_model(test_question)
print("Bigger (2.8B) finetuned model (test): ", bigger_finetuned_output)

INFO:lamini.api.lamini:Using 3.10 InferenceQueue Interface


Bigger (2.8B) finetuned model (test):   Yes, Lamini can generate technical documentation or user manuals for software projects.


In [42]:
count = 0
for i in range(len(train_dataset)):
 if "keep the discussion relevant to Lamini" in train_dataset[i]["answer"]:
  print(i, train_dataset[i]["question"], train_dataset[i]["answer"])
  count += 1
print(count)

65 Why do we shiver when we're cold? Let’s keep the discussion relevant to Lamini.
69 Why do we dream? Let’s keep the discussion relevant to Lamini.
134 Can lightning strike the same place twice? Let’s keep the discussion relevant to Lamini.
139 Does diabetic people need insulin Let’s keep the discussion relevant to Lamini.
204 Can you get a tan through a window? Let’s keep the discussion relevant to Lamini.
221 Can animals laugh? Let’s keep the discussion relevant to Lamini.
246 Can you taste food without a sense of smell? Let’s keep the discussion relevant to Lamini.
260 what is onestream Let’s keep the discussion relevant to Lamini.
295 Can you live without a sense of smell? Let’s keep the discussion relevant to Lamini.
304 Can you die from a broken heart? Let’s keep the discussion relevant to Lamini.
317 Why do some people have freckles? Let’s keep the discussion relevant to Lamini.
388 Can you tickle yourself? Let’s keep the discussion relevant to Lamini.
413 Why do we blush when 

In [43]:
base_tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-70m")
base_model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-70m")
print(inference("What do you think of Mars?", base_model, base_tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.




I think I’m going to go to the next page.

I think I’m going to go to the next page.

I think I’m going to go to the next page.

I think I’m going to go to the next page.

I think I’m going to go to the next page.

I think I’m going to go to the next page.

I


###Finetuning using lamini

In [45]:
model = BasicModelRunner("EleutherAI/pythia-410m")
model.load_data_from_jsonlines("lamini_docs.jsonl", input_key="question", output_key="answer")
model.train(is_public=True)

INFO:lamini.api.lamini:Using 3.10 InferenceQueue Interface
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://laministorage.blob.core.windows.net/training-data/platform/9fbc3324a1a0888aab9bdbdd22eff7b4f36d3c5c0b8996713d8d0245be8ac8a4?st=REDACTED&se=REDACTED&sp=REDACTED&sv=REDACTED&sr=REDACTED&sig=REDACTED'
Request method: 'HEAD'
Request headers:
    'x-ms-version': 'REDACTED'
    'Accept': 'application/xml'
    'User-Agent': 'azsdk-python-storage-blob/12.19.1 Python/3.10.12 (Linux-6.1.58+-x86_64-with-glibc2.35)'
    'x-ms-date': 'REDACTED'
    'x-ms-client-request-id': 'f2c911bc-f98b-11ee-bcb6-0242ac1c000c'
No body was attached to the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 404
Response headers:
    'Transfer-Encoding': 'chunked'
    'Server': 'Windows-Azure-Blob/1.0 Microsoft-HTTPAPI/2.0'
    'x-ms-request-id': 'cb480a5b-901e-00b7-0e98-8d6a6b000000'
    'x-ms-client-request-id': 'f2c911bc-f98b-11ee-bcb6-0242ac1c000c'
    


Uploading data....


INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 201
Response headers:
    'Content-Length': '0'
    'Server': 'Windows-Azure-Blob/1.0 Microsoft-HTTPAPI/2.0'
    'x-ms-request-id': 'cb480a7d-901e-00b7-2a98-8d6a6b000000'
    'x-ms-client-request-id': 'f3424244-f98b-11ee-bcb6-0242ac1c000c'
    'x-ms-version': 'REDACTED'
    'x-ms-content-crc64': 'REDACTED'
    'x-ms-request-server-encrypted': 'REDACTED'
    'Date': 'Sat, 13 Apr 2024 11:49:57 GMT'
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://laministorage.blob.core.windows.net/training-data/platform/9fbc3324a1a0888aab9bdbdd22eff7b4f36d3c5c0b8996713d8d0245be8ac8a4?comp=REDACTED&st=REDACTED&se=REDACTED&sp=REDACTED&sv=REDACTED&sr=REDACTED&sig=REDACTED'
Request method: 'PUT'
Request headers:
    'Content-Length': '143'
    'If-None-Match': '*'
    'x-ms-version': 'REDACTED'
    'Content-Type': 'application/xml'
    'Accept': 'application/xml'
    'User-Agent': 'azsdk-python-storage-blob/12.

Upload to blob completed for data.
Data pairs uploaded to blob.

Your dataset id is: 9fbc3324a1a0888aab9bdbdd22eff7b4f36d3c5c0b8996713d8d0245be8ac8a4 . Consider using this in the future to train using the same data. 
Eg: llm.train(dataset_id='9fbc3324a1a0888aab9bdbdd22eff7b4f36d3c5c0b8996713d8d0245be8ac8a4')
Training job submitted! Check status of job 5940 here: https://app.lamini.ai/train/5940


{'job_id': 5940,
 'status': 'SCHEDULED',
 'dataset_id': '9fbc3324a1a0888aab9bdbdd22eff7b4f36d3c5c0b8996713d8d0245be8ac8a4'}

In [52]:
from llama import BasicModelRunner

model = BasicModelRunner("EleutherAI/pythia-410m")
model.load_data_from_jsonlines("lamini_docs.jsonl", input_key="question", output_key="answer")
model.train(is_public=True)

INFO:lamini.api.lamini:Using 3.10 InferenceQueue Interface
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://laministorage.blob.core.windows.net/training-data/platform/08e5f6b8690d3dab572bfe29e0ae38757548ffbce7e0c0d5dfc29fabaa57189d?st=REDACTED&se=REDACTED&sp=REDACTED&sv=REDACTED&sr=REDACTED&sig=REDACTED'
Request method: 'HEAD'
Request headers:
    'x-ms-version': 'REDACTED'
    'Accept': 'application/xml'
    'User-Agent': 'azsdk-python-storage-blob/12.19.1 Python/3.10.12 (Linux-6.1.58+-x86_64-with-glibc2.35)'
    'x-ms-date': 'REDACTED'
    'x-ms-client-request-id': 'b3d7233e-f98d-11ee-bcb6-0242ac1c000c'
No body was attached to the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 404
Response headers:
    'Transfer-Encoding': 'chunked'
    'Server': 'Windows-Azure-Blob/1.0 Microsoft-HTTPAPI/2.0'
    'x-ms-request-id': '99c642b3-601e-009c-409a-8deaa7000000'
    'x-ms-client-request-id': 'b3d7233e-f98d-11ee-bcb6-0242ac1c000c'
    


Uploading data....


INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 201
Response headers:
    'Content-Length': '0'
    'Server': 'Windows-Azure-Blob/1.0 Microsoft-HTTPAPI/2.0'
    'x-ms-request-id': '99c642d9-601e-009c-5d9a-8deaa7000000'
    'x-ms-client-request-id': 'b44922a4-f98d-11ee-bcb6-0242ac1c000c'
    'x-ms-version': 'REDACTED'
    'x-ms-content-crc64': 'REDACTED'
    'x-ms-request-server-encrypted': 'REDACTED'
    'Date': 'Sat, 13 Apr 2024 12:02:30 GMT'
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://laministorage.blob.core.windows.net/training-data/platform/08e5f6b8690d3dab572bfe29e0ae38757548ffbce7e0c0d5dfc29fabaa57189d?comp=REDACTED&st=REDACTED&se=REDACTED&sp=REDACTED&sv=REDACTED&sr=REDACTED&sig=REDACTED'
Request method: 'PUT'
Request headers:
    'Content-Length': '143'
    'If-None-Match': '*'
    'x-ms-version': 'REDACTED'
    'Content-Type': 'application/xml'
    'Accept': 'application/xml'
    'User-Agent': 'azsdk-python-storage-blob/12.

Upload to blob completed for data.
Data pairs uploaded to blob.

Your dataset id is: 08e5f6b8690d3dab572bfe29e0ae38757548ffbce7e0c0d5dfc29fabaa57189d . Consider using this in the future to train using the same data. 
Eg: llm.train(dataset_id='08e5f6b8690d3dab572bfe29e0ae38757548ffbce7e0c0d5dfc29fabaa57189d')
Training job submitted! Check status of job 5942 here: https://app.lamini.ai/train/5942


{'job_id': 5942,
 'status': 'SCHEDULED',
 'dataset_id': '08e5f6b8690d3dab572bfe29e0ae38757548ffbce7e0c0d5dfc29fabaa57189d'}

In [58]:
model.job_id = 5940
out = model.evaluate()

In [59]:
lofd = []
for e in out['eval_results']:
    q  = f"{e['input']}"
    at = f"{e['outputs'][0]['output']}"
    ab = f"{e['outputs'][1]['output']}"
    di = {'question': q, 'trained model': at, 'Base Model' : ab}
    lofd.append(di)
df = pd.DataFrame.from_dict(lofd)
style_df = df.style.set_properties(**{'text-align': 'left'})
style_df = style_df.set_properties(**{"vertical-align": "text-top"})
style_df

,question,trained model,Base Model
0,Does the documentation have a secret code that unlocks a hidden treasure?,Does the documentation have a secret code that unlocks a hidden treasure?,"I'm trying to find a way to unlock a secret code in the codebook. A: I'm not sure if this is the best way, but I found a way to do it. I'm not sure if this is the best way, but I found a way. I found a way to do it. I found a way to do it. I found a way to do it. I found a way to do it. I found a way to do it. I found a way to do it. I found a way to do it. I found a way to do it. I found a way to do it. I found a way to do it. I found a way to do it. I found a way to do it. I found a way to do it. I found a way to do it. I found a way to do it. I found a way to do it. I found a way to do it. I found a way to do it. I found a way to do it."
1,Does Lamini support named entity recognition and extraction?,"Yes, Lamini supports named entity recognition and extraction. It can be used to extract and recognize entities from text data. It can be used to extract and recognize entities from text data. It can be used to extract and recognize entities from text data. It can be used to extract and recognize entities from text data. It can be used to extract and recognize entities from text data. It can be used to extract and recognize entities from text data. It can be used to extract and recognize entities from text data. It can be used to extract and recognize entities from text data. It can be used to extract and recognize entities from text data. It can be used to extract and recognize entities from text data. It can be used to extract and recognize entities from text data. It can be used to extract and recognize entities from text data. It can be used to extract and recognize entities from text data. It can be used to extract and recognize entities from text data. It can be used to extract and recognize entities from text data. It can be used to extract and recognize entities from text data. It can be used to extract and recognize entities from text data. It can be used to extract and recognize entities from text data. It can be used to extract and recognize entities from",I am trying to implement a web application that uses Lamini. I am using the following code: import lamini.core.api.api.model.entity.entity.Entity; import lamini.core.api.api.api.model.entity.entity.EntityType; import lamini.core.api.api.api.model.entity.entity.EntityType.EntityType; import lamini.core.api.api.api.model.entity.entity.EntityType.EntityType; import lamini.core.api.api.api.model.entity.entity.EntityType.EntityType; import lamini.core.api.api.api.model.entity.entity.EntityType.EntityType; import lamini.core.api.api.api.model.entity.entity.EntityType.EntityType; import lamini.core.api.api.api.model.entity.entity.EntityType.EntityType; import lamini.core.api.api.api.model.entity.entity.EntityType.EntityType; import lamini.core.
2,Does Lamini have the ability to understand and generate regular expressions?,"Yes, Lamini has the ability to understand and generate regular expressions. It can be used to generate regular expressions for any language.","A: I don't think so. A: I don't think so. I'm not sure what you mean by ""regular expressions"" in the context of Lamini. A: I don't think so. I think you mean regular expressions. A: I don't think so. I think you mean regular expressions. A: I think you mean regular expressions. A: I think you mean regular expressions. A: I think you mean regular expressions. A: I think you mean regular expressions. A: I think you mean regular expressions. A: I think you mean regular expressions. A: I think you mean regular expressions. A: I think you mean regular expressions. A: I think you mean regular expressions. A: I think you mean regular expressions. A: I think you mean regular expressions. A: I think you mean regular expressions. A: I think you mean regular expressions"
3,How can we monitor the status of a job using the `check_job_status()